In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import random
import re
import os
import math
import time

In [2]:

## Functions to facilitate webscraping
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    else:
        return True

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
]

def select_agent():
    n=random.randint(0,len(user_agents)-1)
    return user_agents[n]

In [3]:
users = pd.read_csv('dutch_users.tsv',sep='\t')
users = users[users['nr_reviews']<500]

In [4]:
for i,row in users.iterrows():
    
    if not os.path.isfile(os.path.join('Reviews',f'{row["id"]}.txt')):
        time.sleep(5)

        per_page = 20
        nr_pages = math.ceil(int(row['nr_reviews'])/per_page)
        #print(f'{row["nr_reviews"]} reviews in total.' )
        user_id = row['id']

        reviews = []
        base_url = 'https://www.goodreads.com/review/list/'

        for i in range(1,(nr_pages+1)):

            url = f'{base_url}{user_id}?utf8=%E2%9C%93&order=d&sort=review&page={i}&view=reviews&per_page={per_page}'
            print(url)

            headers = {'User-Agent': select_agent() }
            response = requests.get( url, headers=headers)

            if response:
                html_page = response.text
                soup = BeautifulSoup(html_page,"lxml")

                links = soup.findAll('a')
                for a in links:
                    href = a.get('href')
                    if re.search(r'\/review\/show',str(href)):
                        reviews.append(href)

            reviews = list(set(reviews))
            print(len(reviews))

        print(f'Total: {len(reviews)}')

        path = os.path.join('Reviews',f'{user_id}.txt')  

        if not os.path.exists(path):
            out = open(path,'w',encoding='utf-8')

            for review in reviews:

                review_url = 'https://www.goodreads.com'+review
                print(review_url)

                headers = {'User-Agent': select_agent() }
                response = requests.get( review_url, headers=headers)
                if response:
                    html_page = response.text
                    soup = BeautifulSoup(html_page,"lxml")

                    div_review = soup.find('div',{'class':'reviewText'})
                    if div_review is None:
                         div_review = soup.find('div',{'class':'big450BoxContent'})
                    if div_review is not None:   
                        full_text = div_review.text.strip()
                        full_text = re.sub(r'\s+', ' ',full_text)
                        out.write(full_text)
                        out.write('\n\n')
                    else:
                        print(f'Problem with {review_url}')

            out.close()

https://www.goodreads.com/review/list/145266663?utf8=%E2%9C%93&order=d&sort=review&page=1&view=reviews&per_page=20
20
https://www.goodreads.com/review/list/145266663?utf8=%E2%9C%93&order=d&sort=review&page=2&view=reviews&per_page=20
40
https://www.goodreads.com/review/list/145266663?utf8=%E2%9C%93&order=d&sort=review&page=3&view=reviews&per_page=20
60
https://www.goodreads.com/review/list/145266663?utf8=%E2%9C%93&order=d&sort=review&page=4&view=reviews&per_page=20
80
https://www.goodreads.com/review/list/145266663?utf8=%E2%9C%93&order=d&sort=review&page=5&view=reviews&per_page=20
100
https://www.goodreads.com/review/list/145266663?utf8=%E2%9C%93&order=d&sort=review&page=6&view=reviews&per_page=20
120
https://www.goodreads.com/review/list/145266663?utf8=%E2%9C%93&order=d&sort=review&page=7&view=reviews&per_page=20
140
https://www.goodreads.com/review/list/145266663?utf8=%E2%9C%93&order=d&sort=review&page=8&view=reviews&per_page=20
160
https://www.goodreads.com/review/list/145266663?utf8

https://www.goodreads.com/review/show/5125315359
https://www.goodreads.com/review/show/5455086131
https://www.goodreads.com/review/show/5237556830
https://www.goodreads.com/review/show/4586184590
https://www.goodreads.com/review/show/4530947765
https://www.goodreads.com/review/show/4967501650
https://www.goodreads.com/review/show/5295449253
https://www.goodreads.com/review/show/5665103972
https://www.goodreads.com/review/show/4730681268
https://www.goodreads.com/review/show/4737927442
https://www.goodreads.com/review/show/4605023751
https://www.goodreads.com/review/show/4969969855
https://www.goodreads.com/review/show/4434479563
https://www.goodreads.com/review/show/4434498669
https://www.goodreads.com/review/show/4679519696
https://www.goodreads.com/review/show/5046164820
https://www.goodreads.com/review/show/5225530390
https://www.goodreads.com/review/show/4923672916
https://www.goodreads.com/review/show/4434528381
https://www.goodreads.com/review/show/4764662252
https://www.goodread

https://www.goodreads.com/review/show/5823822107
https://www.goodreads.com/review/show/6430110045
https://www.goodreads.com/review/show/5664319912
https://www.goodreads.com/review/show/6177475890
https://www.goodreads.com/review/show/6505534270
https://www.goodreads.com/review/show/5230728326
https://www.goodreads.com/review/show/6433378488
https://www.goodreads.com/review/show/4379857275
https://www.goodreads.com/review/show/5384268394
https://www.goodreads.com/review/show/5166262418
https://www.goodreads.com/review/show/4387572557
https://www.goodreads.com/review/show/4597133718
https://www.goodreads.com/review/show/5602263930
https://www.goodreads.com/review/show/4387582216
https://www.goodreads.com/review/show/5755101439
https://www.goodreads.com/review/show/5463110983
https://www.goodreads.com/review/list/56834629?utf8=%E2%9C%93&order=d&sort=review&page=1&view=reviews&per_page=20
20
https://www.goodreads.com/review/list/56834629?utf8=%E2%9C%93&order=d&sort=review&page=2&view=revie

https://www.goodreads.com/review/show/3339504438
https://www.goodreads.com/review/show/1669636063
https://www.goodreads.com/review/show/1949974242
https://www.goodreads.com/review/show/6038093156
https://www.goodreads.com/review/show/1669650660
https://www.goodreads.com/review/show/3998730211
https://www.goodreads.com/review/show/3466370694
https://www.goodreads.com/review/show/1669641923
https://www.goodreads.com/review/show/2681890790
https://www.goodreads.com/review/show/1804694110
https://www.goodreads.com/review/show/3647861584
https://www.goodreads.com/review/show/3130825815
https://www.goodreads.com/review/show/1669641642
https://www.goodreads.com/review/show/1669640877
https://www.goodreads.com/review/show/5683957007
https://www.goodreads.com/review/show/2681893485
https://www.goodreads.com/review/show/1669636012
https://www.goodreads.com/review/show/3675429850
https://www.goodreads.com/review/show/3386355661
https://www.goodreads.com/review/show/1669635950
https://www.goodread

https://www.goodreads.com/review/show/2449565622
https://www.goodreads.com/review/show/4749236210
https://www.goodreads.com/review/show/4040474289
https://www.goodreads.com/review/show/6242934827
https://www.goodreads.com/review/show/4892862208
https://www.goodreads.com/review/show/4763672869
https://www.goodreads.com/review/show/6243063124
https://www.goodreads.com/review/show/4244021258
https://www.goodreads.com/review/show/3510421852
https://www.goodreads.com/review/show/1773985021
https://www.goodreads.com/review/show/2870472100
https://www.goodreads.com/review/show/4729642869
https://www.goodreads.com/review/show/3289030514
https://www.goodreads.com/review/show/5572826288
https://www.goodreads.com/review/show/3629982371
https://www.goodreads.com/review/show/4584230543
https://www.goodreads.com/review/show/4742198166
https://www.goodreads.com/review/show/2951037334
https://www.goodreads.com/review/show/3775145287
https://www.goodreads.com/review/show/2121486317
https://www.goodread

...

https://www.goodreads.com/review/show/3996897223
https://www.goodreads.com/review/show/3553503528
https://www.goodreads.com/review/show/3909867257
https://www.goodreads.com/review/show/4578750554
https://www.goodreads.com/review/show/5598254601
https://www.goodreads.com/review/show/3579832101
https://www.goodreads.com/review/show/4229551971
https://www.goodreads.com/review/show/5429227167
https://www.goodreads.com/review/show/3094455413
https://www.goodreads.com/review/show/3598296512
https://www.goodreads.com/review/show/3687367548
https://www.goodreads.com/review/show/4840714617
https://www.goodreads.com/review/show/3860747949
https://www.goodreads.com/review/show/3449052096
https://www.goodreads.com/review/show/3890199219
https://www.goodreads.com/review/show/4195660164
https://www.goodreads.com/review/show/3279920643
https://www.goodreads.com/review/show/5014365257
https://www.goodreads.com/review/show/3860742605
https://www.goodreads.com/review/show/6345700994
https://www.goodread

https://www.goodreads.com/review/show/2927672749
https://www.goodreads.com/review/show/2927642379
https://www.goodreads.com/review/show/2927639237
https://www.goodreads.com/review/show/2927664255
https://www.goodreads.com/review/show/2927654600
https://www.goodreads.com/review/show/2927639656
https://www.goodreads.com/review/show/2927642735
https://www.goodreads.com/review/show/2927647569
https://www.goodreads.com/review/show/2927645345
https://www.goodreads.com/review/show/2927642511
https://www.goodreads.com/review/show/2927642913
https://www.goodreads.com/review/show/2927666218
https://www.goodreads.com/review/show/2927640354
https://www.goodreads.com/review/show/2927639314
https://www.goodreads.com/review/show/2927656064
https://www.goodreads.com/review/show/2927656386
https://www.goodreads.com/review/show/2927641775
https://www.goodreads.com/review/show/2927642933
https://www.goodreads.com/review/show/2927642479
https://www.goodreads.com/review/show/2927655599
https://www.goodread

https://www.goodreads.com/review/show/3718009831
https://www.goodreads.com/review/show/3718012299
https://www.goodreads.com/review/show/5594639937
https://www.goodreads.com/review/show/4649433238
https://www.goodreads.com/review/show/3718021225
https://www.goodreads.com/review/show/5183406278
https://www.goodreads.com/review/show/5547762598
https://www.goodreads.com/review/show/4488724571
https://www.goodreads.com/review/show/6335073513
https://www.goodreads.com/review/show/5547757464
https://www.goodreads.com/review/show/3717931692
https://www.goodreads.com/review/show/4102561595
https://www.goodreads.com/review/show/5470534347
https://www.goodreads.com/review/show/4102595036
https://www.goodreads.com/review/show/3718020836
https://www.goodreads.com/review/show/3717927117
https://www.goodreads.com/review/show/6449920854
https://www.goodreads.com/review/show/6362978102
https://www.goodreads.com/review/show/4566761415
https://www.goodreads.com/review/list/96047362?utf8=%E2%9C%93&order=d

https://www.goodreads.com/review/show/4345726040
https://www.goodreads.com/review/show/3706402270
https://www.goodreads.com/review/show/4354718511
https://www.goodreads.com/review/show/3959858345
https://www.goodreads.com/review/show/3092725327
https://www.goodreads.com/review/show/4098000045
https://www.goodreads.com/review/show/4295759027
https://www.goodreads.com/review/show/3131179460
https://www.goodreads.com/review/show/4700567145
https://www.goodreads.com/review/show/2964538673
https://www.goodreads.com/review/show/4934582542
https://www.goodreads.com/review/show/2818441271
https://www.goodreads.com/review/show/3128941663
https://www.goodreads.com/review/show/3030744156
https://www.goodreads.com/review/show/6051906319
https://www.goodreads.com/review/show/6129244298
https://www.goodreads.com/review/show/5941196761
https://www.goodreads.com/review/show/2835859184
https://www.goodreads.com/review/show/5229101102
https://www.goodreads.com/review/show/3202140798
https://www.goodread

ConnectionError: HTTPSConnectionPool(host='www.goodreads.com', port=443): Max retries exceeded with url: /review/show/3487593836 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe2a8b91510>: Failed to establish a new connection: [Errno 60] Operation timed out'))